In [16]:
#clear all variables
%reset -f

In [17]:
import requests
import pdfplumber
import pandas as pd

# Get the PDF
with pdfplumber.open("../input/2010.pdf") as pdf:
    page = pdf.pages[0]
    text = page.extract_text()
    print(text)

Menu Detail by Ward
2010 Menu - Ward: 01
CDOT
Alley Resurface Menu Humps Blocks EstCost
N CAMPBELL AV--W DIVERSEY AV--N MAPLEWOOD AV--W GEORGE ST 0.04 1,129
Alley Resurface Menu 0.04 1,129
Alley Speed Hump Menu Humps EstCost
W POTOMAC AV--W EVERGREEN AV--N WOLCOTT AV--N MILWAUKEE AV 5 2,625
Alley Speed Hump Menu 5 2,625
Curb/Gutter Menu CG_Blks SW Blks EstCost
N BELL AV-- W LE MOYNE ST (1500 N) -to- W HIRSCH ST (1400 N) 4,817
Curb/Gutter Menu 4,817
Curb/Gutter Menu - Change Order CG_Blks SW Blks EstCost
1420-8 N. Bell 5,967
Curb/Gutter Menu - Change Order 5,967
Miscellaneous-CDOT UnitCount EstCost
Across Damen at Erie 1.00 700
Damen/Division/Logan Blvd/Milwaukee-multiple locations 4.00 55,300
Miscellaneous-CDOT 5.00 56,000
Sidewalk Menu CG_Blks SW Blks EstCost
W BELDEN AV-- N WESTERN AV (2400 W) -to- N CAMPBELL AV (2450 W) 23,263
W HOMER ST-- N CAMPBELL AV (2500 W) -to- N ROCKWELL ST (2600 W) 59,986
W WABANSIA AV-- N OAKLEY AV (2300 W) -to- N OAKLEY AV (2300 W) 8,819
Sidewalk Menu 92,0

In [21]:

df = pd.DataFrame(columns=["ward", "type", "location", "cg_blks", "sw_blks", "humps","blocks", "unitcount", "estcost"])
gatherflag = False
typeflag = False
#set all variables to empty strings
[ward, location, type, CG_blks, SW_blks, humps, unitcount, estcost] = [""]*8
variables = [ward, location, type, CG_blks, SW_blks, humps, unitcount, estcost]
for line in text.splitlines():
    if "Ward" in line:
        ward = line.split()[-1]
    if "EstCost" in line:
        gatherflag = True
        typeflag = True
    if "Printed" in line:
        gatherflag = False
    elif not any(char.isdigit() for char in line):
        gather = False   
    #if typeflag is true, then extract the text of the line into variable type
    if typeflag:
        type = line
        table_heading = line
        #remove the strings " CG_Blks" " SW Blks" from the type variable
        quantity_strings= [" CG_Blks", " SW_Blks", " Humps", " Blocks", " UnitCount", " EstCost"]
        for string in quantity_strings:
            type = type.replace(string, "")

        typeflag = False
    #if gatherflag is true and line doesn't contain type, then extract the last number of the line into a variable estcost
    elif gatherflag and not type in line:
        #assert all variables are empty strings
        print(type)
        assert all(var == "" for var in variables)
        estcost = line.split()[-1]
        if "CG Blks" in type and "SW Blks" in table_heading:
            SW_blks = line.split()[-2]
            CG_blks = line.split()[-3]
            if not SW_blks.isdigit():
                SW_blks = 'Missing'
            if not CG_blks.isdigit():
                CG_blks = 'Missing'
            humps = 'NA'
            blocks = 'NA'
            unitcount = 'NA'
        elif "Humps" in table_heading and "Blocks" in table_heading:
            humps = line.split()[-3]
            blocks = line.split()[-2]
            if not blocks.isdigit():
                blocks = 'Missing'
            if not humps.isdigit():
                humps = 'Missing'
            unitcount = 'NA'
            CG_blks = 'NA'
            SW_blks = 'NA'
        elif "Humps" in table_heading:
            humps = line.split()[-2]
            unitcount = 'NA'
            CG_blks = 'NA'
            SW_blks = 'NA'
            blocks = 'NA'
        elif "Blocks" in table_heading:
            blocks = line.split()[-2]
            unitcount = 'NA'
            CG_blks = 'NA'
            SW_blks = 'NA'
            humps = 'NA'
        elif "UnitCount" in table_heading:
            unitcount = line.split()[-2]
            CG_blks = 'NA'
            SW_blks = 'NA'
            humps = 'NA'
            blocks = 'NA'
        #set location equal to the line replacing all the variables with empty strings
        location = line
        for variable in variables:
            location = location.replace(variable, "")
        print(location)
        #append the variables to the dataframe
        df = df.append({"ward": ward, "type": type, "location": location, "cg_blks": CG_blks, "sw_blks": SW_blks, "humps": humps, "blocks": blocks, "unitcount": unitcount, "estcost": estcost}, ignore_index=True)
        #reset all variables to empty strings
        [location, CG_blks, SW_blks, humps, unitcount, estcost] = [""]*8
#output the dataframe to a csv
df.to_csv("../output/2010.csv", index=False)


Alley Resurface Menu
N CAMPBELL AV--W DIVERSEY AV--N MAPLEWOOD AV--W GEORGE ST 0.04 1,129


/tmp/ipykernel_10122/3029872925.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"ward": ward, "type": type, "location": location, "cg_blks": CG_blks, "sw_blks": SW_blks, "humps": humps, "blocks": blocks, "unitcount": unitcount, "estcost": estcost}, ignore_index=True)


ValueError: too many values to unpack (expected 6)